In [66]:
import pandas as pd
import numpy as np

In [67]:
data = pd.read_csv("./train.csv", encoding='big5')

In [68]:
data = data.iloc[:, 3:]

In [69]:
data[data == 'NR'] = 0
raw_data = data.to_numpy()

In [70]:
month_data = {}
for month in range(12):
    sample = np.empty([18, 20 * 24])
    for day in range(20):
        sample[:, 24 * day:24 * (day + 1)] = raw_data[18 * (20 * month + day):18 * (20 * month + day + 1), :]
    month_data[month] = sample

In [71]:
raw_data[0].shape

(24,)

In [72]:
x = np.empty([12 * 471, 18 * 9], dtype=float)
y = np.empty([12 * 471, 1], dtype=float)

for month in range(12):
    for day in range(20):
        for hour in range(24):
            if day == 19 and hour > 14:
                continue
            x[month * 471 + day * 24 + hour, :] = month_data[month][:, day * 24 + hour : day * 24 + hour + 9].reshape(1, -1)  # 变一维
            y[month * 471 + day * 24 + hour, 0] = month_data[month][9, day * 24 + hour + 9]

In [73]:
mean_x = np.mean(x, axis=0)
std_x = np.std(x, axis=0)
for i in range(len(x)):
    for j in range(len(x[0])):
        if std_x[j] != 0:
            x[i][j] = (x[i][j] - mean_x[j]) / std_x[j]

In [74]:
import math
x_train_set = x[: math.floor(len(x) * 0.8), :]
y_train_set = y[: math.floor(len(y) * 0.8), :]
x_validation = x[math.floor(len(x) * 0.8):, :]
y_validation = y[math.floor(len(y) * 0.8):, :]

In [75]:
dim = 18 * 9 + 1
w = np.zeros([dim, 1])
x = np.concatenate((np.ones([12 * 471, 1]), x), axis=1).astype(float)
learning_rate = 100
iter_time = 1000
adagrad = np.zeros([dim, 1])
eps = 0.0000000001
for t in range(iter_time):
    loss = np.sqrt(np.sum(np.power(np.dot(x, w) - y, 2)) / 471 / 12)
    if t % 100 == 0:
        print(str(t) + ":" + str(loss))
    gradient = 2 * np.dot(x.transpose(), np.dot(x, w) - y)
    adagrad += gradient ** 2
    w = w - learning_rate * gradient / np.sqrt(adagrad + eps)
np.save('weight.npy', w)

0:27.071214829194115
100:33.78905859777455
200:19.9137512981971
300:13.531068193689686
400:10.645466158446165
500:9.277353455475058
600:8.518042045956495
700:8.014061987588413
800:7.636756824775685
900:7.336563740371118


In [77]:
testdata = pd.read_csv('./test.csv',header=None, encoding='big5')
test_data = testdata.iloc[:, 2:]
test_data[test_data == 'NR'] = 0
test_data = test_data.to_numpy()
test_x = np.empty([240, 18 * 9], dtype=float)

for i in range(240):
    test_x[i, :] = test_data[18 * i:18 * (i + 1), :].reshape(1, -1)
    
mean_x = np.mean(test_x, axis=0)
std_x = np.std(test_x, axis=0)
for i in range(len(test_x)):
    for j in range(len(test_x[0])):
        if std_x[j] != 0:
            test_x[i][j] = (test_x[i][j] - mean_x[j]) / std_x[j]
            
test_x = np.concatenate((np.ones([240, 1]), test_x), axis=1).astype(float)

<ipython-input-77-ad65b3ab28f3>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data[test_data == 'NR'] = 0
D:\Anaconda\anaconda\lib\site-packages\pandas\core\frame.py:3089: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._where(-key, value, inplace=True)


In [78]:
w = np.load('weight.npy')
ans_y = np.dot(test_x, w)

In [79]:
import csv
with open('submit.csv', mode='w', newline='') as submit_file:
    csv_writer = csv.writer(submit_file)
    header = ['id', 'value']
    print(header)
    csv_writer.writerow(header)
    for i in range(240):
        row = ['id_' + str(i), ans_y[i][0]]
        csv_writer.writerow(row)
        print(row)

['id', 'value']
['id_0', 4.678193301920871]
['id_1', 16.627075223850795]
['id_2', 12.573452355132176]
['id_3', 11.766726450991513]
['id_4', 19.713056009198297]
['id_5', 16.041245051271538]
['id_6', 15.987136486937862]
['id_7', 24.619806096705872]
['id_8', 7.688414586466813]
['id_9', 52.831288535322]
['id_10', 17.443158186018543]
['id_11', 12.558944471834332]
['id_12', 53.638844202552036]
['id_13', 34.7878590123183]
['id_14', 12.145375454850331]
['id_15', 6.8371401732941575]
['id_16', 22.805792122256157]
['id_17', 55.101650765653645]
['id_18', -6.8847547058217415]
['id_19', 13.701312809984978]
['id_20', 29.37878280508567]
['id_21', 56.30604445152599]
['id_22', 4.564775561562946]
['id_23', 13.295201067496798]
['id_24', 11.797055277127406]
['id_25', 27.07474795903242]
['id_26', 14.1900658503111]
['id_27', 59.01327628045754]
['id_28', 9.546946572495386]
['id_29', 44.76286607135952]
['id_30', 19.554702165132365]
['id_31', 0.8945263072065526]
['id_32', 4.629574808853434]
['id_33', 20.5174018